In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
tokenizer = Tokenizer()

In [4]:
data = """I walked through the door with you \n The air was cold\n But something about it felt like home somehow
\nAnd I left my scarf there at your sisters house
\nAnd youve still got it in your drawer even now
\n Oh your sweet disposition
\n And my wide-eyed gaze
\n We are singing in the car getting lost upstate
\n Autumn leaves falling down like pieces into place
\n And I can picture it after all these days
\n And I know its long gone and that magics not here no more
\n And I might be okay but Im not fine at all\n
Cause there we are again on that little town street \n
You almost ran the red  cause you were lookin  over at me \n
Wind in my hair I was there \n
I remember it all too well \n
Photo album on the counter \n
Your cheeks were turning red \n
You used to be a little kid with glasses in a twin-sized bed \n
And your mother s telling stories  bout you on the tee-ball team \n
You told me  bout your past thinking your future was me \n
And I know it s long gone and there was nothing else I could do \n
And I forget about you long enough to forget why I needed to \n
 Cause there we are again in the middle of the night \n
We re dancing  round the kitchen in the refrigerator light \n
Down the stairs I was there \n
I remember it all too well yeah \n
And maybe we got lost in translation \n
Maybe I asked for too much \n
But maybe this thing was a masterpiece  til you tore it all up \n
Running scared I was there \n
I remember it all too well \n
And you call me up again just to break me like a promise \n
So casually cruel in the name of being honest \n
I m a crumpled up piece of paper lying here \n
 Cause I remember it all all all \n
Too well \n
Time won t fly it s like I m paralyzed by it \n
I d like to be my old self again \n
But I m still trying to find it \n
After plaid shirt days and nights when you made me your own \n
Now you mail back my things and I walk home alone \n
But you keep my old scarf from that very first week \n
 Cause it reminds you of innocence \n
And it smells like me \n
You can t get rid of it \n
 Cause you remember it all too well yeah \n
 Cause there we are again when I loved you so \n
Back before you lost the one real thing you ve ever known \n
It was rare I was there I remember it all too well \n
Wind in my hair you were there you remember it all \n
Down the stair you were there you remember it all \n
It was rare I was there I remember it all too well"""

In [5]:
corpus = data.lower().split('\n')

In [6]:
tokenizer.fit_on_texts(corpus)

In [7]:
total_words = len(tokenizer.word_index) + 1

In [8]:
#Make Sequence of every Sequence

In [10]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
   n_gram_sequence = token_list[:i+1]
   input_sequences.append(n_gram_sequence)
print(input_sequences[:5])

[[1, 65], [1, 65, 66], [1, 65, 66, 5], [1, 65, 66, 5, 67], [1, 65, 66, 5, 67, 37]]


In [11]:
# Now pad them into a regular shape, use pre padding

In [13]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
 maxlen=max_sequence_len, padding='pre'))


In [14]:
# Now input will be the Sequence and Output will the last element in the Sequence
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

In [16]:
# Encode the output/labels as the output will be too many will require too many Neurons as you want to predict many words
ys = tf.keras.utils.to_categorical(labels,num_classes = total_words)

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Bidirectional,Dense
model = Sequential()
model.add(Embedding(total_words, 8))
model.add(Bidirectional(LSTM(max_sequence_len-1)))
model.add(Dense(total_words, activation='softmax'))


In [24]:
model.compile(loss= 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'] )

In [26]:
history = model.fit(xs,ys,epochs = 1500,verbose = 1)

Epoch 1/1500
14/14 [==============================] - 0s 12ms/step - loss: 4.8807 - accuracy: 0.0495
Epoch 2/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.8179 - accuracy: 0.0495
Epoch 3/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.7937 - accuracy: 0.0495
Epoch 4/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.7741 - accuracy: 0.0495
Epoch 5/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.7587 - accuracy: 0.0472
Epoch 6/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.7465 - accuracy: 0.0519
Epoch 7/1500
14/14 [==============================] - 0s 11ms/step - loss: 4.7301 - accuracy: 0.0519
Epoch 8/1500
14/14 [==============================] - 0s 12ms/step - loss: 4.7161 - accuracy: 0.0542
Epoch 9/1500
14/14 [==============================] - 0s 12ms/step - loss: 4.7029 - accuracy: 0.0542
Epoch 10/1500
14/14 [==============================] - 0s 12ms/step - loss: 4.6912 - accura

In [27]:
print(history)

In [29]:
seed_text = 'I walked through the door'

In [30]:
token_seed = tokenizer.texts_to_sequences(seed_text)

In [33]:
first_element = token_seed[0]

In [34]:
first_element

[1]

In [36]:
pad_seed = pad_sequences([first_element],maxlen =max_sequence_len-1,padding = 'pre')

In [39]:
new_word = np.argmax(model.predict(pad_seed),axis = 1)

In [40]:
new_word

array([10])

In [41]:
word_get = tokenizer.word_index

In [43]:
for key,value in word_get.items():
  if word_get[key]==new_word:
    print(key)

remember


In [44]:
# Doing it N number of times

In [55]:
seed_text = 'I left my scarf'
next_words = 200

In [56]:
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding = 'pre')
  predicted = np.argmax(model.predict(token_list,verbose = 0),axis = 1)
  output_word = ""
  for word,index in tokenizer.word_index.items():
    if(index ==predicted):
      output_word = word
      break
  seed_text+=" "+output_word
print(seed_text)


I left my scarf there at your sisters house disposition disposition do hair like like to hair you you being even bed bed bed bed a future rare like like i bed hair a twin do do bed bed to why else i could do stairs like like like hair like like hair like long at like to on the promise of the night promise cause i to hair me like hair like like to hair me like hair like like to on to all these do bed bed that promise promise do hair like like hair like like a disposition do bed hair like like in again a ever known cheeks disposition cheeks disposition promise promise needed rare like like rare like like rare like like i about was lookin i keep home disposition disposition disposition own do hair like like hair like like a disposition do bed hair like like in again a ever known cheeks disposition cheeks disposition promise promise needed rare like like rare like like rare like like i about was lookin i keep home disposition disposition disposition own do hair like like hair like like a d